In [1]:
import numpy as np;
import matplotlib as plot;
import matplotlib.pyplot as plt;

import random;

In [ ]:
"""
SudokuBoard class
Representação de um tabuleiro de Sudoku com 9x9 posições, em algum momento do jogo.
"""
class SudokuBoard:
    def __init__(self, board):
        self.board = board;
        
    def __str__(self):
        string = "";
        for row in self.board:
            string += " ".join(map(str, row)) + "\n";
        return string;
    
    def __eq__(self, other):
        return self.board == other.board;
    
    def __hash__(self):
        return hash(tuple(map(tuple, self.board)));
    
    def copy(self):
        return SudokuBoard([row[:] for row in self.board]);
    
    def count_conflicts(self):
        conflicts = 0;
        for i in range(9):
            for j in range(9):
                if self.board[i][j] != 0:
                    for k in range(9):
                        if k != j and self.board[i][k] == self.board[i][j]:
                            conflicts += 1;
                        if k != i and self.board[k][j] == self.board[i][j]:
                            conflicts += 1;
                        if i // 3 == k // 3 and j // 3 == k % 3 and self.board[k][j] == self.board[i][j]:
                            conflicts += 1;
        return conflicts;
    
    def evaluate(self) -> float:
        """
        Calcula o número de conflitos em colunas e subgrades 3x3.
        Quanto menor o valor, melhor.
        """
        conflicts = 0;
        state = self.board.copy();
        # Conflitos em linhas
        for row in range(9):
            conflicts += len(state[row]) - len(set(state[row]))
            
        # Conflitos em colunas
        for col in range(9):
            column = [state[row][col] for row in range(9)]
            conflicts += len(column) - len(set(column))
        # Conflitos em subgrades 3x3
        for i in range(0, 9, 3):
            for j in range(0, 9, 3):
                subgrid = []
                for x in range(3):
                    for y in range(3):
                        subgrid.append(state[i + x][j + y])
                conflicts += len(subgrid) - len(set(subgrid))
        return conflicts
    
    def is_goal(self):
        return self.count_conflicts() == 0;
    
board : SudokuBoard = SudokuBoard([[1, 2, 3, 4, 5, 6, 7, 8, 9],
                                  [4, 5, 6, 7, 8, 9, 1, 2, 3],
                                  [7, 8, 9, 1, 2, 3, 4, 5, 6],
                                  [2, 3, 4, 5, 6, 7, 8, 9, 1],
                                  [5, 6, 7, 8, 9, 1, 2, 3, 4],
                                  [8, 9, 1, 2, 3, 4, 5, 6, 7],
                                  [3, 4, 5, 6, 7, 8, 9, 1, 2],
                                  [6, 7, 8, 9, 1, 2, 3, 4, 5],
                                  [9, 1, 2, 3, 4, 5, 6, 7, 8]]);

print(board);

print(board.evaluate());
print(board.is_goal());

1 2 3 4 5 6 7 8 9
4 5 6 7 8 9 1 2 3
7 8 9 1 2 3 4 5 6
2 3 4 5 6 7 8 9 1
5 6 7 8 9 1 2 3 4
8 9 1 2 3 4 5 6 7
3 4 5 6 7 8 9 1 2
6 7 8 9 1 2 3 4 5
9 1 2 3 4 5 6 7 8

27
False


In [4]:
"""
Solver para Sudoku
"""

from copy import deepcopy;
from abc import ABCMeta, abstractmethod

class SudokuSolver(metaclass=ABCMeta):
    def __init__(self, board):
        self.board = board;
        self.unassigned = [(i, j) for i in range(9) for j in range(9) if board.board[i][j] == 0];
        self.unassigned.sort(key=lambda x: board.board[x[0]][x[1]]);
        self.fixed_cells = [(i, j) for i in range(9) for j in range(9) if board.board[i][j] != 0];
        
    def getNeighbors(self, state: SudokuBoard) -> set[SudokuBoard]:
        """
        Gera vizinhos permutando duas células não fixas na mesma linha.
        """
        neighbors = set()
        for row in range(9):
            # Encontra células não fixas na linha
            mutable_cols = [col for col in range(9) if (row, col) not in self.fixed_cells]
            if len(mutable_cols) < 2:
                continue
            # Gera permutações
            for _ in range(5):  # Limita para 5 permutações por linha (ajustável)
                i, j = random.sample(mutable_cols, 2)
                neighbor = deepcopy(state)
                neighbor[row][i], neighbor[row][j] = neighbor[row][j], neighbor[row][i]
                neighbors.add(tuple(map(tuple, neighbor)))  # Para hash
        # Converte de volta para lista de listas
        return [list(map(list, neighbor)) for neighbor in neighbors];
    
    def evaluate(self) -> float:
        return self.board.evaluate();
    
    @abstractmethod
    def solve(self) -> SudokuBoard:
        pass;

##  Busca Local
### Caminhada Aleatória (Random Walk)

$$
\large
N(s) \neq \emptyset, ~~~~ P_s(s) = \frac{1}{|N_s|}
$$


In [ ]:
ASSA

In [ ]:
from random import choice;

class RandomWalkSolver(SudokuSolver):
    def getNeighbors(self, state):
        return super().getNeighbors(state);
    
    def solve(self, state, niter : int = 1000):
        best_state = state;
        
        while(niter):
            next_state : SudokuBoard = choice(self.getNeighbors(state));
            if next_state.board.evaluate() > best_state.board.evaluate():
                best_state = next_state;
            niter -= 1;
            
            if niter % 100 == 0: print(f"{niter}");
        
        return best_state;
    
RWS : RandomWalkSolver = RandomWalkSolver(board);
RWS.solve(board, 1000);
